In [ ]:
import pandas as pd
import sys
sys.path.append("..")
from helpers.db_util import db, ReqResp, DirectTest, ProbeTest, RetroTest, Url
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
db.init("sanity_check")
db.connect()

# General

In [ ]:
u = pd.DataFrame(Url.select().dicts())
r = pd.DataFrame(ReqResp.select().dicts())

In [ ]:
def parse_url(row):
    url = row["real_url"]
    split = url.split("/")
    return split[-2], split[-1]

r[["function", "valid"]] = r.apply(parse_url, axis=1, result_type="expand")

In [ ]:
print("URLs with error or notes from mitmproxy: e.g., unexpected data received that is not passed to the functions")
r.loc[(r["msg"] != "") | (r["error"] != "")].sort_values(by=["function", "valid"])[["id", "function", "valid", "error", "msg", "req_type"]].head(5)

# Probe/Proxy Tests

In [ ]:
pt = pd.DataFrame(ProbeTest.select().dicts())

In [ ]:
print("General stats on all collected proxy testcases")
pt[["type", "violation", "extra"]].value_counts().to_frame()

In [ ]:
res = pd.merge(r.loc[r["probe_id"].isna()], pt, left_on="url", right_on="url")

In [ ]:
res[["resp_code", "resp_version", "type", "violation"]].value_counts().to_frame().head(20)

In [ ]:
print("Number of unique tests:", len(set(res["function"])))
print("Number of unique tests (results collected):", len(set(res["name"])))
print("Tests without results:",  set(res["function"]) - set(res["name"])) 

In [ ]:
san = res.loc[res["name"] == res["function"]]

In [ ]:
san[["valid", "type", "violation"]].value_counts().to_frame()

In [ ]:
for valid, type, violation in san[["valid", "type", "violation"]].value_counts().index:
    print(valid, type, violation)
    df = san.loc[(san["valid"] == valid) & (san["type"] == type) & (san["violation"] == violation)]
    print(df["name"].unique().tolist())
    print()
    
# invalid -> follows specification
# field_value_start_or_end_with_whitespace: problem wptserver strips the leading whitespaces
# transfer_encoding_http11: problem mitmproxy does not allow the invalid transfer-encoding

# All the other results are expected

# Direct tests

In [ ]:
# Check direct tests
dt = pd.DataFrame(DirectTest.select().dicts())
dt = pd.merge(dt, r, on="url", how="inner")
dt[["function", "valid"]] = dt.apply(parse_url, axis=1, result_type="expand")

In [ ]:
dt.loc[dt["name"] == "content_length_2XX_connect"]["real_url"]

In [ ]:
for valid, type, violation in dt[["valid", "type", "violation"]].value_counts().index:
    print(valid, type, violation)
    df = dt.loc[(dt["valid"] == valid) & (dt["type"] == type) & (dt["violation"] == violation)]
    print(df["name"].unique().tolist())
    print()

# No entry for valid/invalid
# content_length_2XX_connect, transfer_encoding_2XX_connect: cannot be tested correctly as WPT does not pass CONNECT option through 

# Incorrect result: valid -> breaks specification
# allow_crlf_start: cannot be tested correctly as WPT closes connection directly; after FIRST CRLF
# code_405_blocked_methods: cannot be tested correctly as WPT return code 200 for CONNECT

# Incorrect result: invalid -> follows specification
# continue_before_upgrade: cannot be tested correctly as WPT returns an automatic response to Expect: 100-Continue

# The other tests have the expected results

# Retro tests

In [ ]:
# check retro tests!
rt = pd.DataFrame(RetroTest.select().dicts())
rt = pd.merge(rt, r, on="url", how="left")
rt[["function", "valid"]] = rt.apply(parse_url, axis=1, result_type="expand")

In [ ]:
# All results are as expected
for valid, type, violation in rt[["valid", "type", "violation"]].value_counts().index:
    print(valid, type, violation)
    df = rt.loc[(rt["valid"] == valid) & (rt["type"] == type) & (rt["violation"] == violation)]
    print(df["name"].unique().tolist())
    print()